In [3]:
# ============================================================
# Block V1 — Distinctiveness: MII ≠ Volume (Robust Column Mapping)
# Paper 3 — Pipeline V (Validation)
#
# PURPOSE
# Demonstrate that the Mobility Infrastructure Index (MII)
# is not reducible to simple mobility volume measures.
#
# INPUT
# - Output from Pipeline A4 (Mobility + MII)
#
# OUTPUTS (saved to Desktop/Output Pipeline V (Validation))
# - V1_mii_vs_volume_metrics.json
# - V1_mii_vs_volume_models.csv
# - V1_quadrant_typology_by_tract.csv.gz
# ============================================================

import os
import json
import time
from datetime import datetime

import numpy as np
import pandas as pd
import statsmodels.api as sm

# -----------------------------
# 0) PATHS (FINAL)
# -----------------------------
MII_INPUT_PATH = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline A (Mobility)/A4/"
    "mobility_by_tract_aug2024_with_mii_FINAL.csv.gz"
)

V_OUT_DIR = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline V (Validation)/"
)

os.makedirs(V_OUT_DIR, exist_ok=True)

# -----------------------------
# 1) HELPERS — ROBUST COLUMN RESOLUTION
# -----------------------------
def _norm(s: str) -> str:
    return s.strip().lower()

def _pick_first(existing_cols, preferred_exact, preferred_contains):
    """
    Choose a column deterministically:
    1) exact match (case-insensitive)
    2) contains match (case-insensitive)
    """
    norm_map = {_norm(c): c for c in existing_cols}

    # exact
    for cand in preferred_exact:
        if _norm(cand) in norm_map:
            return norm_map[_norm(cand)], {"rule": "exact", "match": cand}

    # contains
    existing_norm = [_norm(c) for c in existing_cols]
    for pattern in preferred_contains:
        pattern_n = _norm(pattern)
        for i, c_n in enumerate(existing_norm):
            if pattern_n in c_n:
                return existing_cols[i], {"rule": "contains", "match": pattern}

    return None, {"rule": None, "match": None}

def resolve_volume_columns(df_cols):
    """
    Robustly resolve:
    - ct_id
    - mii
    - visits (raw volume)
    - unique (raw unique visitors)
    """
    # ct_id and mii are stable by your Pipeline S code
    ct_id_col, ct_meta = _pick_first(
        df_cols,
        preferred_exact=["ct_id"],
        preferred_contains=["ct_id", "cd_setor"]
    )

    mii_col, mii_meta = _pick_first(
        df_cols,
        preferred_exact=["mii"],
        preferred_contains=["mii"]
    )

    # Visits candidates: be conservative (prefer totals over rates)
    visits_col, visits_meta = _pick_first(
        df_cols,
        preferred_exact=[
            "total_visits", "visits", "n_visits", "visits_total"
        ],
        preferred_contains=[
            "total_visits", "visits_total", "visits", "nvisits"
        ],
    )

    # Unique candidates: prefer "unique" / "unique_visitors" / "visitors_unique"
    unique_col, unique_meta = _pick_first(
        df_cols,
        preferred_exact=[
            "unique_visitors", "unique", "visitors_unique", "n_unique", "unique_users"
        ],
        preferred_contains=[
            "unique_visitors", "visitors_unique", "unique", "uniq"
        ],
    )

    return {
        "ct_id_col": ct_id_col, "ct_id_meta": ct_meta,
        "mii_col": mii_col, "mii_meta": mii_meta,
        "visits_col": visits_col, "visits_meta": visits_meta,
        "unique_col": unique_col, "unique_meta": unique_meta,
    }

def list_candidates(df_cols, keyword):
    kw = keyword.lower()
    return [c for c in df_cols if kw in c.lower()]

# -----------------------------
# 2) INIT
# -----------------------------
run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
t0 = time.time()

print("▶️ Pipeline V — Block V1: Distinctiveness (MII ≠ Volume)")
print(f"Run ID: {run_id}")
print(f"Input file: {MII_INPUT_PATH}")
print(f"Output dir: {V_OUT_DIR}")
print("-" * 70)

# -----------------------------
# 3) LOAD
# -----------------------------
df = pd.read_csv(MII_INPUT_PATH, compression="gzip", low_memory=False)

print(f"Rows loaded: {len(df):,}")
print(f"Columns loaded: {df.shape[1]:,}")
print("-" * 70)

# -----------------------------
# 4) RESOLVE COLUMNS (ROBUST)
# -----------------------------
cols = df.columns.tolist()
mapping = resolve_volume_columns(cols)

print("Resolved column mapping (auto):")
for k in ["ct_id_col", "mii_col", "visits_col", "unique_col"]:
    print(f"  - {k}: {mapping[k]}  | meta={mapping[k.replace('_col','_meta')]}")
print("-" * 70)

missing_core = [k for k in ["ct_id_col", "mii_col", "visits_col", "unique_col"] if mapping[k] is None]
if missing_core:
    print("❌ Could not resolve required columns automatically.")
    print("Column diagnostics (candidates):")
    print(f"  - columns containing 'visit':  {list_candidates(cols, 'visit')[:50]}")
    print(f"  - columns containing 'unique': {list_candidates(cols, 'unique')[:50]}")
    print(f"  - columns containing 'visitor':{list_candidates(cols, 'visitor')[:50]}")
    print(f"  - columns containing 'ct':     {list_candidates(cols, 'ct')[:50]}")
    raise ValueError(
        f"Missing resolved columns: {missing_core}. "
        "Please inspect the candidate lists printed above."
    )

CT_ID_COL = mapping["ct_id_col"]
MII_COL = mapping["mii_col"]
VISITS_COL = mapping["visits_col"]
UNIQUE_COL = mapping["unique_col"]

# Log columns: prefer existing if present; otherwise create.
# We will look for common log aliases first.
LOG_VISITS_COL = None
LOG_UNIQUE_COL = None

# Try to find existing logs
for cand in ["log_total_visits", "log_visits", "log1p_total_visits", "log1p_visits", "log_visits_total"]:
    if cand in df.columns:
        LOG_VISITS_COL = cand
        break

for cand in ["log_unique_visitors", "log_unique", "log1p_unique_visitors", "log1p_unique", "log_visitors_unique"]:
    if cand in df.columns:
        LOG_UNIQUE_COL = cand
        break

# Create logs if missing
df[MII_COL] = pd.to_numeric(df[MII_COL], errors="coerce")
df[VISITS_COL] = pd.to_numeric(df[VISITS_COL], errors="coerce")
df[UNIQUE_COL] = pd.to_numeric(df[UNIQUE_COL], errors="coerce")

if LOG_VISITS_COL is None:
    LOG_VISITS_COL = "log_visits_auto"
    df[LOG_VISITS_COL] = np.log1p(df[VISITS_COL])

if LOG_UNIQUE_COL is None:
    LOG_UNIQUE_COL = "log_unique_auto"
    df[LOG_UNIQUE_COL] = np.log1p(df[UNIQUE_COL])

# Sanity checks
for c in [LOG_VISITS_COL, LOG_UNIQUE_COL]:
    if np.isinf(df[c]).any():
        raise ValueError(f"❌ Infinite values detected in {c}")

print("Log columns in use:")
print(f"  - {LOG_VISITS_COL} (visits)")
print(f"  - {LOG_UNIQUE_COL} (unique)")
print("-" * 70)

# -----------------------------
# 5) QC — MISSINGNESS + RANGES
# -----------------------------
required_cols = [CT_ID_COL, MII_COL, VISITS_COL, UNIQUE_COL, LOG_VISITS_COL, LOG_UNIQUE_COL]
qc_missing = {c: int(df[c].isna().sum()) for c in required_cols}

print("QC — Missingness (core cols):")
for k, v in qc_missing.items():
    print(f"  - {k}: {v:,}")

def _safe_stats(s: pd.Series):
    s2 = pd.to_numeric(s, errors="coerce")
    if s2.notna().sum() == 0:
        return {"min": None, "p50": None, "p99": None, "max": None}
    return {
        "min": float(np.nanmin(s2)),
        "p50": float(np.nanpercentile(s2, 50)),
        "p99": float(np.nanpercentile(s2, 99)),
        "max": float(np.nanmax(s2)),
    }

qc_ranges = {c: _safe_stats(df[c]) for c in [MII_COL, VISITS_COL, UNIQUE_COL, LOG_VISITS_COL, LOG_UNIQUE_COL]}

print("-" * 70)
print("QC — Ranges (key variables):")
for c, st in qc_ranges.items():
    print(f"  - {c}: min={st['min']}, p50={st['p50']}, p99={st['p99']}, max={st['max']}")
print("-" * 70)

# -----------------------------
# 6) ANALYSIS A — CORRELATIONS
# -----------------------------
corr_vars = [MII_COL, VISITS_COL, UNIQUE_COL, LOG_VISITS_COL, LOG_UNIQUE_COL]
corr_df = df[corr_vars].dropna()

pearson_corr = corr_df.corr(method="pearson")
spearman_corr = corr_df.corr(method="spearman")

print("Correlation diagnostics:")
print(f"  Pearson corr(MII, log(visits))  = {pearson_corr.loc[MII_COL, LOG_VISITS_COL]:.4f}")
print(f"  Spearman corr(MII, log(visits)) = {spearman_corr.loc[MII_COL, LOG_VISITS_COL]:.4f}")
print("-" * 70)

# -----------------------------
# 7) ANALYSIS B — HOW MUCH DOES VOLUME EXPLAIN MII?
# -----------------------------
reg_df = df[[MII_COL, LOG_VISITS_COL, LOG_UNIQUE_COL]].dropna()

y = reg_df[MII_COL]
X1 = sm.add_constant(reg_df[[LOG_VISITS_COL]])
X2 = sm.add_constant(reg_df[[LOG_VISITS_COL, LOG_UNIQUE_COL]])

m1 = sm.OLS(y, X1).fit()
m2 = sm.OLS(y, X2).fit()

models_out = pd.DataFrame([
    {
        "model": "MII ~ log(visits)",
        "n_obs": int(m1.nobs),
        "r2": float(m1.rsquared),
        "adj_r2": float(m1.rsquared_adj),
        "beta_log_visits": float(m1.params[LOG_VISITS_COL]),
        "p_log_visits": float(m1.pvalues[LOG_VISITS_COL]),
    },
    {
        "model": "MII ~ log(visits) + log(unique)",
        "n_obs": int(m2.nobs),
        "r2": float(m2.rsquared),
        "adj_r2": float(m2.rsquared_adj),
        "beta_log_visits": float(m2.params[LOG_VISITS_COL]),
        "p_log_visits": float(m2.pvalues[LOG_VISITS_COL]),
        "beta_log_unique": float(m2.params[LOG_UNIQUE_COL]),
        "p_log_unique": float(m2.pvalues[LOG_UNIQUE_COL]),
    }
])

print("Incremental explanation of MII by volume:")
print(models_out.to_string(index=False))
print("-" * 70)

# -----------------------------
# 8) QUADRANT TYPOLOGY — VOLUME × INFRASTRUCTURE
# -----------------------------
quad_df = df[[CT_ID_COL, MII_COL, LOG_VISITS_COL]].dropna()

mii_median = quad_df[MII_COL].median()
vol_median = quad_df[LOG_VISITS_COL].median()

def assign_quadrant(row):
    if row[MII_COL] >= mii_median and row[LOG_VISITS_COL] >= vol_median:
        return "High volume / High infrastructure"
    if row[MII_COL] < mii_median and row[LOG_VISITS_COL] >= vol_median:
        return "High volume / Low infrastructure"
    if row[MII_COL] >= mii_median and row[LOG_VISITS_COL] < vol_median:
        return "Low volume / High infrastructure"
    return "Low volume / Low infrastructure"

quad_df["quadrant"] = quad_df.apply(assign_quadrant, axis=1)

quad_counts = quad_df["quadrant"].value_counts().to_dict()

print("Quadrant distribution:")
for k, v in quad_counts.items():
    print(f"  - {k}: {v:,}")
print("-" * 70)

# -----------------------------
# 9) SAVE OUTPUTS
# -----------------------------
metrics = {
    "run_id": run_id,
    "input_path": MII_INPUT_PATH,
    "rows_total": int(len(df)),
    "resolved_columns": {
        "ct_id": CT_ID_COL,
        "mii": MII_COL,
        "visits": VISITS_COL,
        "unique": UNIQUE_COL,
        "log_visits": LOG_VISITS_COL,
        "log_unique": LOG_UNIQUE_COL,
        "resolution_meta": {
            "ct_id": mapping["ct_id_meta"],
            "mii": mapping["mii_meta"],
            "visits": mapping["visits_meta"],
            "unique": mapping["unique_meta"],
        }
    },
    "missing_required": qc_missing,
    "ranges": qc_ranges,
    "pearson_corr": {"mii_log_visits": float(pearson_corr.loc[MII_COL, LOG_VISITS_COL])},
    "spearman_corr": {"mii_log_visits": float(spearman_corr.loc[MII_COL, LOG_VISITS_COL])},
    "models": models_out.to_dict(orient="records"),
    "quadrant_counts": quad_counts,
    "runtime_seconds": round(time.time() - t0, 2),
}

with open(os.path.join(V_OUT_DIR, "V1_mii_vs_volume_metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)

models_out.to_csv(os.path.join(V_OUT_DIR, "V1_mii_vs_volume_models.csv"), index=False)

quad_df.to_csv(
    os.path.join(V_OUT_DIR, "V1_quadrant_typology_by_tract.csv.gz"),
    index=False,
    compression="gzip"
)

print("✅ Block V1 completed successfully.")
print(f"Outputs saved in: {V_OUT_DIR}")
print(f"Runtime: {metrics['runtime_seconds']} seconds")

▶️ Pipeline V — Block V1: Distinctiveness (MII ≠ Volume)
Run ID: 20260103_185556
Input file: /Users/rafaelalbuquerque/Desktop/Output Pipeline A (Mobility)/A4/mobility_by_tract_aug2024_with_mii_FINAL.csv.gz
Output dir: /Users/rafaelalbuquerque/Desktop/Output Pipeline V (Validation)/
----------------------------------------------------------------------
Rows loaded: 436,868
Columns loaded: 48
----------------------------------------------------------------------
Resolved column mapping (auto):
  - ct_id_col: ct_id  | meta={'rule': 'exact', 'match': 'ct_id'}
  - mii_col: mii  | meta={'rule': 'exact', 'match': 'mii'}
  - visits_col: visits  | meta={'rule': 'exact', 'match': 'visits'}
  - unique_col: unique  | meta={'rule': 'exact', 'match': 'unique'}
----------------------------------------------------------------------
Log columns in use:
  - log_visits_auto (visits)
  - log_unique_auto (unique)
----------------------------------------------------------------------
QC — Missingness (core 

In [5]:
# ============================================================
# Pipeline V — Block V2
# Ablation & Internal Validity of the MII (Robust Version)
# ============================================================

import os
import json
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
INPUT_PATH = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline A (Mobility)/A4/"
    "mobility_by_tract_aug2024_with_mii_FINAL.csv.gz"
)

OUTPUT_DIR = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline V (Validation)/V2"
)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------------------------
# LOAD
# ------------------------------------------------------------
df = pd.read_csv(INPUT_PATH, compression="gzip", low_memory=False)
cols = df.columns.tolist()

# ------------------------------------------------------------
# HELPER — COLUMN RESOLUTION (CONCEPT-DRIVEN)
# ------------------------------------------------------------
def find_col(preferred_exact, preferred_contains):
    for c in preferred_exact:
        if c in cols:
            return c, {"rule": "exact", "match": c}
    for pat in preferred_contains:
        for c in cols:
            if pat.lower() in c.lower():
                return c, {"rule": "contains", "match": pat}
    return None, None

resolved = {}

# Core
resolved["mii"], _ = find_col(["mii"], ["mii"])
resolved["visits"], _ = find_col(
    ["visits", "total_visits"],
    ["visits"]
)
resolved["unique"], _ = find_col(
    ["unique", "unique_visitors"],
    ["unique"]
)

# Permanence
resolved["dwell"], _ = find_col(
    ["dwell_time", "avg_dwell_time", "mean_dwell_time"],
    ["dwell"]
)

# Recurrence
resolved["repeat"], _ = find_col(
    ["repeat_visits", "repeat"],
    ["repeat"]
)
resolved["new"], _ = find_col(
    ["new_visitors", "new"],
    ["new"]
)

# Stability
resolved["stability"], _ = find_col(
    ["stability_cv", "cv_weekly", "weekly_cv"],
    ["stability", "cv"]
)

# ------------------------------------------------------------
# CHECK RESOLUTION
# ------------------------------------------------------------
missing = [k for k, v in resolved.items() if v is None and k != "mii"]

print("Resolved component mapping:")
for k, v in resolved.items():
    print(f"  - {k}: {v}")

if resolved["mii"] is None:
    raise ValueError("❌ Column 'mii' not found — Pipeline A4b integrity violated.")

if missing:
    print("\n❌ Missing conceptual components:")
    for k in missing:
        print(f"  - {k}")
    print("\nAvailable columns:")
    print(cols)
    raise ValueError("Ablation cannot proceed without resolving conceptual components.")

# ------------------------------------------------------------
# HELPER — Z-MEAN
# ------------------------------------------------------------
def zmean(cols):
    Z = StandardScaler().fit_transform(df[cols])
    return Z.mean(axis=1)

# ------------------------------------------------------------
# CONSTRUCT ABLATION INDICES
# ------------------------------------------------------------
df["idx_volume_only"] = zmean([resolved["visits"]])

df["idx_volume_unique"] = zmean([
    resolved["visits"],
    resolved["unique"]
])

df["idx_volume_dwell"] = zmean([
    resolved["visits"],
    resolved["dwell"]
])

df["idx_no_stability"] = zmean([
    resolved["visits"],
    resolved["unique"],
    resolved["dwell"],
    resolved["repeat"],
    resolved["new"]
])

df["idx_no_recurrence"] = zmean([
    resolved["visits"],
    resolved["unique"],
    resolved["dwell"],
    resolved["stability"]
])

# ------------------------------------------------------------
# CORRELATION WITH TRUE MII
# ------------------------------------------------------------
indices = [
    "idx_volume_only",
    "idx_volume_unique",
    "idx_volume_dwell",
    "idx_no_stability",
    "idx_no_recurrence"
]

corr_results = []
for idx in indices:
    rho, _ = spearmanr(df[idx], df[resolved["mii"]])
    corr_results.append({
        "index": idx,
        "spearman_corr_with_mii": round(float(rho), 4)
    })

corr_df = pd.DataFrame(corr_results)
corr_df.to_csv(
    os.path.join(OUTPUT_DIR, "V2_ablation_correlations.csv"),
    index=False
)

# ------------------------------------------------------------
# RANK STABILITY — TOP 10%
# ------------------------------------------------------------
def top_overlap(a, b, q=0.9):
    ta = set(df.loc[df[a] >= df[a].quantile(q)].index)
    tb = set(df.loc[df[b] >= df[b].quantile(q)].index)
    return len(ta & tb) / len(ta)

rank_results = []
for idx in indices:
    overlap = top_overlap(resolved["mii"], idx)
    rank_results.append({
        "index": idx,
        "top10_overlap_with_mii": round(float(overlap), 4)
    })

rank_df = pd.DataFrame(rank_results)
rank_df.to_csv(
    os.path.join(OUTPUT_DIR, "V2_rank_stability.csv"),
    index=False
)

# ------------------------------------------------------------
# QC REPORT
# ------------------------------------------------------------
qc = {
    "n_rows": int(len(df)),
    "resolved_columns": resolved,
    "indices_tested": indices,
    "interpretation_rule": (
        "Low correlation and low top-rank overlap for simplified indices "
        "indicate that the full infrastructural architecture of the MII "
        "is empirically necessary."
    )
}

with open(os.path.join(OUTPUT_DIR, "V2_ablation_qc.json"), "w") as f:
    json.dump(qc, f, indent=2)

print("✅ Block V2 completed successfully.")
print(f"Outputs saved in: {OUTPUT_DIR}")

Resolved component mapping:
  - mii: mii
  - visits: visits
  - unique: unique
  - dwell: dwell_time_mins
  - repeat: repeat_visitors
  - new: new_visitors
  - stability: stability_visits_week_cv_A4
✅ Block V2 completed successfully.
Outputs saved in: /Users/rafaelalbuquerque/Desktop/Output Pipeline V (Validation)/V2


In [6]:
# ============================================================
# Pipeline V — Block V3
# Reliability & Stability of the Mobility Infrastructure Index
# ============================================================

import os
import json
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
INPUT_PATH = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline A (Mobility)/A4/"
    "mobility_by_tract_aug2024_with_mii_FINAL.csv.gz"
)

OUTPUT_DIR = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline V (Validation)/V3"
)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------------------------
# LOAD
# ------------------------------------------------------------
df = pd.read_csv(INPUT_PATH, compression="gzip", low_memory=False)

# ------------------------------------------------------------
# COMPONENTS (already validated in V2)
# ------------------------------------------------------------
components = {
    "visits": "visits",
    "unique": "unique",
    "dwell": "dwell_time_mins",
    "repeat": "repeat_visitors",
    "new": "new_visitors",
    "stability": "stability_visits_week_cv_A4"
}

for k, c in components.items():
    if c not in df.columns:
        raise ValueError(f"Required component missing: {c}")

if "mii" not in df.columns:
    raise ValueError("Column 'mii' not found.")

# ------------------------------------------------------------
# HELPER FUNCTIONS
# ------------------------------------------------------------
def zmean(cols):
    Z = StandardScaler().fit_transform(df[cols])
    return Z.mean(axis=1)

def top_overlap(a, b, q=0.9):
    ta = set(df.loc[df[a] >= df[a].quantile(q)].index)
    tb = set(df.loc[df[b] >= df[b].quantile(q)].index)
    return len(ta & tb) / len(ta)

# ------------------------------------------------------------
# BASELINE (FULL STRUCTURE, NO PCA)
# ------------------------------------------------------------
full_cols = list(components.values())
df["mii_zmean_full"] = zmean(full_cols)

# ------------------------------------------------------------
# PERTURBATION TESTS — DROP ONE COMPONENT
# ------------------------------------------------------------
results_corr = []
results_rank = []

for drop_key, drop_col in components.items():
    cols_used = [c for c in full_cols if c != drop_col]
    label = f"drop_{drop_key}"

    df[label] = zmean(cols_used)

    rho, _ = spearmanr(df["mii"], df[label])
    overlap = top_overlap("mii", label)

    results_corr.append({
        "perturbation": label,
        "spearman_corr_with_mii": round(float(rho), 4)
    })

    results_rank.append({
        "perturbation": label,
        "top10_overlap_with_mii": round(float(overlap), 4)
    })

# ------------------------------------------------------------
# GLOBAL RELIABILITY SUMMARY
# ------------------------------------------------------------
summary = {
    "mean_spearman_corr": float(
        np.mean([r["spearman_corr_with_mii"] for r in results_corr])
    ),
    "min_spearman_corr": float(
        np.min([r["spearman_corr_with_mii"] for r in results_corr])
    ),
    "mean_top10_overlap": float(
        np.mean([r["top10_overlap_with_mii"] for r in results_rank])
    ),
    "min_top10_overlap": float(
        np.min([r["top10_overlap_with_mii"] for r in results_rank])
    ),
}

# ------------------------------------------------------------
# SAVE OUTPUTS
# ------------------------------------------------------------
corr_df = pd.DataFrame(results_corr)
corr_df.to_csv(
    os.path.join(OUTPUT_DIR, "V3_mii_perturbation_correlations.csv"),
    index=False
)

rank_df = pd.DataFrame(results_rank)
rank_df.to_csv(
    os.path.join(OUTPUT_DIR, "V3_mii_rank_stability.csv"),
    index=False
)

qc = {
    "n_rows": int(len(df)),
    "components": components,
    "perturbations_tested": list(components.keys()),
    "summary": summary,
    "interpretation_rule": (
        "High correlation and rank overlap under component perturbation "
        "indicate that the MII behaves as a stable infrastructural signal "
        "rather than a fragile composite."
    )
}

with open(os.path.join(OUTPUT_DIR, "V3_mii_reliability_qc.json"), "w") as f:
    json.dump(qc, f, indent=2)

print("✅ Block V3 completed successfully.")
print(f"Outputs saved in: {OUTPUT_DIR}")

✅ Block V3 completed successfully.
Outputs saved in: /Users/rafaelalbuquerque/Desktop/Output Pipeline V (Validation)/V3


In [9]:
# ============================================================
# Pipeline V — Block V4
# External Validity using CNEFE (IBGE 2022)
# Robust version with tqdm and safe overlap
# ============================================================

import os
import zipfile
import json
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
MII_PATH = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline A (Mobility)/A4/"
    "mobility_by_tract_aug2024_with_mii_FINAL.csv.gz"
)

CNEFE_DIR = "/Users/rafaelalbuquerque/Desktop/Arquivos_CNEFE"

OUTPUT_DIR = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline V (Validation)/V4"
)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------------------------
# LOAD MII
# ------------------------------------------------------------
mii_df = pd.read_csv(
    MII_PATH,
    compression="gzip",
    dtype={"ct_id": str},
    usecols=["ct_id", "mii"]
)

# ------------------------------------------------------------
# HELPERS
# ------------------------------------------------------------
def resolve_ct_col(columns):
    """
    Resolve census tract id column semantically.
    """
    for c in columns:
        cl = c.lower()
        if ("setor" in cl) and ("cd" in cl or "cod" in cl):
            return c
    return None


def try_read(file_obj):
    """
    Try multiple separators; return df or None.
    """
    for sep in [";", "|", "\t", ","]:
        try:
            df = pd.read_csv(
                file_obj,
                sep=sep,
                dtype=str,
                low_memory=False
            )
            if df.shape[1] > 1:
                return df
        except Exception:
            continue
    return None


def top_overlap_safe(df, a, b, q=0.9):
    """
    Safe top-q overlap (returns NaN if undefined).
    """
    ta = set(df.loc[df[a] >= df[a].quantile(q)].index)
    tb = set(df.loc[df[b] >= df[b].quantile(q)].index)

    if len(ta) == 0:
        return np.nan

    return len(ta & tb) / len(ta)

# ------------------------------------------------------------
# LOAD + AGGREGATE CNEFE
# ------------------------------------------------------------
cnefe_counts = {}

zip_files = sorted([f for f in os.listdir(CNEFE_DIR) if f.endswith(".zip")])

print(f"[INFO] Processing {len(zip_files)} CNEFE ZIP files...")

for z in tqdm(zip_files, desc="UF ZIPs"):
    zpath = os.path.join(CNEFE_DIR, z)

    with zipfile.ZipFile(zpath, "r") as zipf:
        inner_files = [
            f for f in zipf.namelist()
            if f.lower().endswith((".csv", ".txt", ".dat"))
        ]

        for fname in inner_files:
            with zipf.open(fname) as f:
                df = try_read(f)
                if df is None:
                    continue

                ct_col = resolve_ct_col(df.columns)
                if ct_col is None:
                    continue

                # aggregate counts
                for ct in df[ct_col].dropna():
                    cnefe_counts[ct] = cnefe_counts.get(ct, 0) + 1

# ------------------------------------------------------------
# BUILD AGGREGATED DATAFRAME
# ------------------------------------------------------------
if not cnefe_counts:
    raise ValueError(
        "❌ No CNEFE records aggregated. "
        "Check file formats or column naming."
    )

cnefe_df = (
    pd.DataFrame.from_dict(
        cnefe_counts,
        orient="index",
        columns=["n_addresses"]
    )
    .reset_index()
    .rename(columns={"index": "ct_id"})
)

cnefe_df.to_csv(
    os.path.join(OUTPUT_DIR, "V4_cnefe_aggregation_by_tract.csv.gz"),
    index=False,
    compression="gzip"
)

# ------------------------------------------------------------
# MERGE WITH MII
# ------------------------------------------------------------
merged = mii_df.merge(cnefe_df, on="ct_id", how="inner")
merged["log_addresses"] = np.log1p(merged["n_addresses"])

# ------------------------------------------------------------
# CORRELATION TESTS
# ------------------------------------------------------------
rho_raw, _ = spearmanr(merged["mii"], merged["n_addresses"])
rho_log, _ = spearmanr(merged["mii"], merged["log_addresses"])

# ------------------------------------------------------------
# TOP 10% OVERLAP (SAFE)
# ------------------------------------------------------------
overlap_raw = top_overlap_safe(merged, "mii", "n_addresses", q=0.9)
overlap_log = top_overlap_safe(merged, "mii", "log_addresses", q=0.9)

# ------------------------------------------------------------
# SAVE RESULTS
# ------------------------------------------------------------
results = pd.DataFrame([
    {
        "metric": "addresses_raw",
        "spearman_corr": round(float(rho_raw), 4),
        "top10_overlap": (
            None if np.isnan(overlap_raw) else round(float(overlap_raw), 4)
        ),
    },
    {
        "metric": "addresses_log",
        "spearman_corr": round(float(rho_log), 4),
        "top10_overlap": (
            None if np.isnan(overlap_log) else round(float(overlap_log), 4)
        ),
    },
])

results.to_csv(
    os.path.join(OUTPUT_DIR, "V4_mii_vs_cnefe_correlations.csv"),
    index=False
)

qc = {
    "n_mii_tracts": int(len(mii_df)),
    "n_cnefe_tracts": int(len(cnefe_df)),
    "n_merged": int(len(merged)),
    "zip_files_processed": zip_files,
    "overlap_definition": "Top-q overlap; undefined (NaN) if no tract exceeds q-quantile",
    "interpretation_rule": (
        "A positive monotonic association between MII and "
        "CNEFE-based address density supports external validity "
        "of the index as a proxy for market infrastructure."
    ),
}

with open(os.path.join(OUTPUT_DIR, "V4_mii_vs_cnefe_qc.json"), "w") as f:
    json.dump(qc, f, indent=2)

print("✅ Block V4 completed successfully.")
print(f"Outputs saved in: {OUTPUT_DIR}")

[INFO] Processing 27 CNEFE ZIP files...


UF ZIPs: 100%|██████████| 27/27 [10:33<00:00, 23.47s/it]


✅ Block V4 completed successfully.
Outputs saved in: /Users/rafaelalbuquerque/Desktop/Output Pipeline V (Validation)/V4


In [3]:
# ============================================================
# Pipeline V — Block V4b
# External Validity using VIIRS Nighttime Lights (Annual, masked)
# Zonal statistics at census tract level
# ============================================================

import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.stats import spearmanr
from tqdm import tqdm
import rasterio
from exactextract import exact_extract

# ------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------

TRACTS_GPKG = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline S (Shapefiles)/S2/"
    "census_tracts_brazil_mobility_mii.gpkg"
)

NIGHTLIGHTS_TIF = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Nightlights/"
    "VIIRS_annual_2024.tif"
)

OUTPUT_DIR = (
    "/Users/rafaelalbuquerque/Desktop/"
    "Output Pipeline V (Validation)/V4b"
)
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUT_ZONAL = os.path.join(
    OUTPUT_DIR, "V4b_nightlights_by_tract.csv.gz"
)
OUT_CORR = os.path.join(
    OUTPUT_DIR, "V4b_mii_vs_nightlights_correlations.csv"
)
OUT_QC = os.path.join(
    OUTPUT_DIR, "V4b_mii_vs_nightlights_qc.json"
)

print("[INFO] Block V4b starting — External validity via nightlights")
print(f" - Tracts GPKG : {TRACTS_GPKG}")
print(f" - Raster     : {NIGHTLIGHTS_TIF}")
print(f" - Output dir : {OUTPUT_DIR}")

# ------------------------------------------------------------
# STEP 1 — LOAD TRACTS WITH MII
# ------------------------------------------------------------

print("\n[STEP 1/5] Loading census tracts with MII...")

gdf = gpd.read_file(TRACTS_GPKG)

required_cols = ["ct_id", "mii", "geometry"]
missing = [c for c in required_cols if c not in gdf.columns]
if missing:
    raise ValueError(f"Missing required columns in GPKG: {missing}")

gdf["ct_id"] = gdf["ct_id"].astype(str)

print(f"[INFO] Tracts loaded: {len(gdf):,}")
print(f"[INFO] CRS (tracts): {gdf.crs}")

# ------------------------------------------------------------
# STEP 2 — OPEN RASTER & ALIGN CRS
# ------------------------------------------------------------

print("\n[STEP 2/5] Opening nightlights raster...")

with rasterio.open(NIGHTLIGHTS_TIF) as src:
    raster_crs = src.crs
    nodata = src.nodata

print(f"[INFO] Raster CRS: {raster_crs}")

if gdf.crs != raster_crs:
    print("[INFO] Reprojecting tracts to raster CRS...")
    gdf = gdf.to_crs(raster_crs)

# ------------------------------------------------------------
# STEP 3 — ZONAL STATISTICS (exactextract)
# ------------------------------------------------------------

print("\n[STEP 3/5] Computing zonal statistics (mean, median, count)...")

stats = exact_extract(
    NIGHTLIGHTS_TIF,
    gdf,
    ["mean", "median", "count"],
    output="pandas"
)

zonal_df = pd.DataFrame({
    "ct_id": gdf["ct_id"].values,
    "mii": gdf["mii"].values,
    "nl_mean": stats["mean"].values,
    "nl_median": stats["median"].values,
    "nl_count": stats["count"].values
})

# ------------------------------------------------------------
# STEP 4 — QC & PREP
# ------------------------------------------------------------

print("\n[STEP 4/5] QC and preparation...")

zonal_df["nl_mean"] = pd.to_numeric(zonal_df["nl_mean"], errors="coerce")
zonal_df["nl_median"] = pd.to_numeric(zonal_df["nl_median"], errors="coerce")
zonal_df["nl_count"] = pd.to_numeric(zonal_df["nl_count"], errors="coerce")

n_total = len(zonal_df)
n_valid = zonal_df["nl_mean"].notna().sum()
n_zero_pixels = (zonal_df["nl_count"].fillna(0) == 0).sum()

zonal_df = zonal_df[zonal_df["nl_mean"].notna()].copy()
zonal_df["log_nl_mean"] = np.log1p(zonal_df["nl_mean"])

print(f"[INFO] Tracts total          : {n_total:,}")
print(f"[INFO] Tracts with NL signal : {n_valid:,}")
print(f"[INFO] Tracts with zero pix  : {n_zero_pixels:,}")

# ------------------------------------------------------------
# STEP 5 — CORRELATION & OVERLAP
# ------------------------------------------------------------

print("\n[STEP 5/5] Correlation tests...")

rho_raw, _ = spearmanr(zonal_df["mii"], zonal_df["nl_mean"])
rho_log, _ = spearmanr(zonal_df["mii"], zonal_df["log_nl_mean"])

def top_overlap_safe(df, a, b, q=0.9):
    ta = set(df.loc[df[a] >= df[a].quantile(q)].index)
    tb = set(df.loc[df[b] >= df[b].quantile(q)].index)
    if len(ta) == 0:
        return np.nan
    return len(ta & tb) / len(ta)

overlap_raw = top_overlap_safe(zonal_df, "mii", "nl_mean", q=0.9)
overlap_log = top_overlap_safe(zonal_df, "mii", "log_nl_mean", q=0.9)

# ------------------------------------------------------------
# SAVE OUTPUTS
# ------------------------------------------------------------

print("\n[SAVE] Writing outputs...")

zonal_df.to_csv(
    OUT_ZONAL,
    index=False,
    compression="gzip"
)

corr_df = pd.DataFrame([
    {
        "metric": "nightlights_raw",
        "spearman_corr": round(float(rho_raw), 4),
        "top10_overlap": (
            None if np.isnan(overlap_raw)
            else round(float(overlap_raw), 4)
        )
    },
    {
        "metric": "nightlights_log",
        "spearman_corr": round(float(rho_log), 4),
        "top10_overlap": (
            None if np.isnan(overlap_log)
            else round(float(overlap_log), 4)
        )
    }
])

corr_df.to_csv(OUT_CORR, index=False)

qc = {
    "engine_used": "exactextract",
    "n_tracts_input": int(n_total),
    "n_tracts_with_signal": int(n_valid),
    "n_zero_pixel_count": int(n_zero_pixels),
    "nightlights_raster": NIGHTLIGHTS_TIF,
    "interpretation_rule": (
        "Positive monotonic association between MII and "
        "nighttime light intensity supports external validity "
        "using an independent physical proxy of sustained "
        "spatial activity."
    )
}

with open(OUT_QC, "w") as f:
    json.dump(qc, f, indent=2)

print("\n[DONE] Block V4b completed successfully.")
print(f" - Zonal stats : {OUT_ZONAL}")
print(f" - Correlation : {OUT_CORR}")
print(f" - QC report  : {OUT_QC}")

[INFO] Block V4b starting — External validity via nightlights
 - Tracts GPKG : /Users/rafaelalbuquerque/Desktop/Output Pipeline S (Shapefiles)/S2/census_tracts_brazil_mobility_mii.gpkg
 - Raster     : /Users/rafaelalbuquerque/Desktop/Nightlights/VIIRS_annual_2024.tif
 - Output dir : /Users/rafaelalbuquerque/Desktop/Output Pipeline V (Validation)/V4b

[STEP 1/5] Loading census tracts with MII...
[INFO] Tracts loaded: 472,780
[INFO] CRS (tracts): EPSG:4674

[STEP 2/5] Opening nightlights raster...
[INFO] Raster CRS: GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
[INFO] Reprojecting tracts to raster CRS...


ERROR 1: PROJ: internal_proj_identify: /opt/anaconda3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 6 is expected. It comes from another PROJ installation.



[STEP 3/5] Computing zonal statistics (mean, median, count)...

[STEP 4/5] QC and preparation...
[INFO] Tracts total          : 472,780
[INFO] Tracts with NL signal : 472,780
[INFO] Tracts with zero pix  : 0

[STEP 5/5] Correlation tests...

[SAVE] Writing outputs...

[DONE] Block V4b completed successfully.
 - Zonal stats : /Users/rafaelalbuquerque/Desktop/Output Pipeline V (Validation)/V4b/V4b_nightlights_by_tract.csv.gz
 - Correlation : /Users/rafaelalbuquerque/Desktop/Output Pipeline V (Validation)/V4b/V4b_mii_vs_nightlights_correlations.csv
 - QC report  : /Users/rafaelalbuquerque/Desktop/Output Pipeline V (Validation)/V4b/V4b_mii_vs_nightlights_qc.json
